# Laser from longitudinal (FROG data) and transverse profile

In this tutorial, you will learn to create a Laser from experimental measurements of the laser spectrum and 2D image of the transverse intensity profile.

In [ ]:
# For VS code
%matplotlib inline
# For jupyter notbook use "%matplotlib ipympl"

Now lets load all the required packages for the tutorial.

In [ ]:
from lasy.laser import Laser
from lasy.profiles.combined_profile import CombinedLongitudinalTransverseProfile
from lasy.profiles.longitudinal import LongitudinalProfileFromData
from lasy.profiles.transverse import TransverseProfileFromData
from lasy.profiles.transverse.hermite_gaussian_profile import HermiteGaussianTransverseProfile
from lasy.utils.mode_decomposition import hermite_gauss_decomposition
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import numpy as np
import skimage                  # Required for the image processing.

Next, let's define the physical parameters defining our laser pulse. 

In [ ]:
polarization    = (1, 0)        # Linearly polarized in the x direction
energy_J        = 1             # Pulse energy in Joules
cal             = 0.2e-6        # Pixel size calibration term

## Reconstruct longitudinal profile

Here as a input file we would be using data recorded using [Frequency-resolved optical grating (FROG)](https://en.wikipedia.org/wiki/Frequency-resolved_optical_gating) techinique. This would provide us with the spectral phase, intensity and frequency (This has been converted to wavelength).

In [ ]:
# Load the data from online datasheet
data_file_csv  = "https://github.com/user-attachments/files/17042670/df_intensity_spectral.csv"

# Read data from the file, in SI units
exp_wavelength = np.loadtxt(data_file_csv, usecols=0, dtype="float")
exp_spectrum   = np.loadtxt(data_file_csv, usecols=1, dtype="float")   # Arbitary units
exp_phase      = np.loadtxt(data_file_csv, usecols=2, dtype="float")

Now, let's fininsh the reconstuction by initializing a LASY LongitudinalProfile from experimentally measured spectrum. The central wavelength used below, is calculated in this step.

In [ ]:
longitudinal_data = {
    "datatype"    : "spectral",
    "axis"        : exp_wavelength,    # Make sure that this parameter is monotonically increasing
    "intensity"   : exp_spectrum,
    "phase"       : exp_phase,         # Radians
    "dt"          : 1e-15,             # Temporal resolution (only required if datatype is spectral)
}

# Create the longitudinal profile
longitudinal_profile      = LongitudinalProfileFromData(
    longitudinal_data, lo = -200e-15, hi = 200e-15
)

## Reconstruct transverse profile

For the following reconstruction the data is provided in the .png format, which can be read using scikit image package.

In [ ]:
# Define the transverse profile of the laser pulse
data_file_img   = "https://user-images.githubusercontent.com/27694869/228038930-d6ab03b1-a726-4b41-a378-5f4a83dc3778.png"
intensity_data  = skimage.io.imread(data_file_img)

# Data cleaning: Remove negative values
intensity_scale = np.max(intensity_data)                    # Maximum value of the intensity
intensity_data[intensity_data < intensity_scale / 100] = 0  # Remove negative values and potential noise.[Recommended: 1% of the maximum value!]

In [ ]:
# Axes
nx, ny          = (intensity_data.shape)
lo              = (0, 0)                             # Lower bound   
hi              = (ny * cal, nx * cal)               # Upper bound

As done earlier for the longitudinal profile, we would construct the transverse profile using extracted data.\
*[This function also centers the data by default]*

In [ ]:
transverse_profile = TransverseProfileFromData(
    intensity_data, [lo[0], lo[1]], [hi[0], hi[1]]
)

In [ ]:
# Plotting the data
fig, ax     = plt.subplots()
cax         = ax.imshow(
    intensity_data,
    aspect  = "auto",
    extent  = [lo[0], hi[0], lo[1], hi[1]],
)

# Add color bar and labels
color_bar   = fig.colorbar(cax)
color_bar.set_label(r"Fluence  (J/cm$^2$)")
ax.set_xlabel(r"x / micron")
ax.set_ylabel(r"y / micron")
plt.show()

## Combine longitudinal and transverse profiles

In [ ]:
raw_laser_profile   = CombinedLongitudinalTransverseProfile(
    wavelength      = longitudinal_profile.lambda0,
    pol             = polarization,
    laser_energy    = energy_J,
    long_profile    = longitudinal_profile,
    trans_profile   = transverse_profile,
)

## Profile denoising
LASY functions can be used for denoising/cleaning. Here, the measured profile is decomposed into Hermite-Gauss modes, and the cleaning is obtained by keeping only the first few modes. 
Take a look at the following [example](https://github.com/LASY-org/lasy/blob/13f0e4515493deca36c1375be1d9e83c7e379d42/examples/example_modal_decomposition_data.py).

In [ ]:
# Maximum Hermite-Gauss mode index in x and y
n_modes_x = 2
n_modes_y = 2

# Calculate the decomposition and waist of the laser pulse
modeCoeffs, waist = hermite_gauss_decomposition(
    transverse_profile, n_x_ma = n_modes_x, n_y_max = n_modes_y, res = cal
)

Construct the filtered profile by summing the first few Hermite-Gauss modes.

In [ ]:
energy_frac = 0
for i, mode_key in enumerate(list(modeCoeffs)):
    tmp_transverse_profile = HermiteGaussianTransverseProfile(
        waist, mode_key[0], mode_key[1]
    )
    energy_frac += modeCoeffs[mode_key] ** 2        # Energy fraction of the mode
    if i == 0:                                      # First mode (0,0)
        laser_profile_cleaned = modeCoeffs[mode_key] * CombinedLongitudinalTransverseProfile(
            longitudinal_profile.lambda0,
            polarization,
            energy_J,
            longitudinal_profile,
            tmp_transverse_profile,
        )
    else:                                           # All other modes
        laser_profile_cleaned += modeCoeffs[mode_key] * CombinedLongitudinalTransverseProfile(
            longitudinal_profile.lambda0,
            polarization,
            energy_J,
            longitudinal_profile,
            tmp_transverse_profile,
        )

# Energy loss due to decomposition
energy_loss = 1 - energy_frac
print(f"Energy loss: {energy_loss * 100:.2f}%")

In [ ]:
# Plot the original and denoised profiles
# Create a grid for plotting
x               = np.linspace(-5 * waist, 5 * waist, 500)
X, Y            = np.meshgrid(x, x)

# Determine the figure parameters
fig, ax         = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)
fig.suptitle(
    "Hermite-Gauss Reconstruction using n_x_max = %i, n_y_max = %i" % (n_x_max, n_y_max)
)

# Plot the original profile
pltextent       = (np.min(x) * 1e6, np.max(x) * 1e6, np.min(x) * 1e6, np.max(x) * 1e6)
prof1           = np.abs(raw_laser_profile.evaluate(X, Y, 0)) ** 2
divider0        = make_axes_locatable(ax[0])
ax0_cb          = divider0.append_axes("right", size="5%", pad=0.05)
pl0             = ax[0].imshow(prof1, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar0           = fig.colorbar(pl0, cax=ax0_cb)
cbar0.set_label("Intensity (norm.)")
ax[0].set_xlabel("x (micron)")
ax[0].set_ylabel("y (micron)")
ax[0].set_title("Original Profile")

# Plot the reconstructed profile
prof2           = np.abs(laser_profile_cleaned.evaluate(X, Y, 0)) ** 2
divider1        = make_axes_locatable(ax[1])
ax1_cb          = divider1.append_axes("right", size="5%", pad=0.05)
pl1             = ax[1].imshow(prof2, cmap="magma", extent=pltextent, vmin=0, vmax=np.max(prof1))
cbar1           = fig.colorbar(pl1, cax=ax1_cb)
cbar1.set_label("Intensity (norm.)")
ax[1].set_xlabel("x (micron)")
ax[1].set_ylabel("y (micron)")
ax[1].set_title("Reconstructed Profile")

# Plot the error
prof3           = (prof1 - prof2) / np.max(prof1)  # Normalized error
divider2        = make_axes_locatable(ax[2])
ax2_cb          = divider2.append_axes("right", size="5%", pad=0.05)
pl2             = ax[2].imshow(100 * np.abs(prof3), cmap="magma", extent=pltextent)
cbar2           = fig.colorbar(pl2, cax=ax2_cb)
cbar2.set_label("|Intensity Error| (%)")
ax[2].set_xlabel("x (micron)")
ax[2].set_ylabel("y (micron)")
ax[2].set_title("Error")

plt.show()

## Create a laser

Now we have done the hard part!\
From the cleaned profile, we can create a LASY Laser object (3D cartesian or cylindrical geometry) and write to a file that can be read from compliant codes!!!\
*Constructing the object using 3D geometry, might take a while to run depending on the hardware you are using.*

In [ ]:
dimensions      = "xyt"                         # Use 3D geometry
lo              = (-40e-6, -20e-6, -50e-15)     # Lower bounds of the simulation box
hi              = (40e-6,   20e-6,  50e-15)     # Upper bounds of the simulation box
num_points      = (300, 300, 200)               # Number of points in each dimension
# num_points    = (50, 50, 20)                  # Low res for quick tests

laser_xyt       = Laser(dimensions, lo, hi, num_points, laser_profile_cleaned)  # Laser
laser_xyt.normalize(energy_J * energy_frac)                                     # Normalize the laser energy
laser_xyt.show()

# Save the laser object to a file
laser_xyt.write_to_file("Laser_xyt_denoised","h5",save_as_vector_potential=True)

In [ ]:
# Create the laser object in cylindrical geometry and propagate it backwards
dimensions  = "rt"                  # Use cylindrical geometry
lo          = (0,    -50e-15)       
hi          = (40e-6, 50e-15)       
num_points  = (500, 400)            

laser       = Laser(dimensions, lo, hi, num_points, laser_profile_cleaned)    
laser.normalize(energy_J * energy_frac)                                 
laser.propagate(-100e-6)            # Propagate the laser pulse backwards
laser.show()

# Save the laser object to a file
laser.write_to_file("Laser_rt_propagated","h5",save_as_vector_potential=True)